В данном блокноте будет реализована задача дообучения LLM  с помощью метода LoRa на датасете Dahoas/sft-gptj-synthetic-prompt-responses.

В датасете около 44 тысяч пар «prompt–response»: в столбце prompt лежит задание или вопрос, который подаётся на вход модели, а в столбце response — желаемый ответ, который мы будем считать таргетом.

Цель - взять относительно небольшую уже предобученную языковую модель, добавить к её слоям LoRA-адаптеры, обучить только эти дополнительные параметры на данном датасете и в итоге получить модель, которая по новому prompt сможет генерировать ответ, максимально похожий на целевой response.

In [1]:
from datasets import load_dataset
import random

In [ ]:
!pip -q install --upgrade transformers accelerate datasets bitsandbytes peft trl sentencepiece einops evaluate
import torch, platform, os, transformers, datasets, peft, bitsandbytes, trl
print('Torch:', torch.__version__)
print('CUDA available?', torch.cuda.is_available())
!nvidia-smi || true

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.8 MB/s eta 0:00:00
Torch: 2.8.0+cu126
CUDA available? False
/bin/bash: line 1: nvidia-smi: command not found


In [2]:
from dataclasses import dataclass

@dataclass
class Config:
    #Базовая модель
    base_model: str = "gpt2" #TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    dataset_name: str = "Dahoas/sft-gptj-synthetic-prompt-responses"
    subset_size: int = 10000        # чтобы обучение шло быстрее

    # QLoRA (4-бит) или обычная LoRA
    use_qlora: bool = True         # True = QLoRA, False = LoRA без квантования

    #Гиперпараметры LoRA
    lora_r: int = 16 #размер скрытого пространства lora
    lora_alpha: int = 32 #для мастабирования вклада lora адаптеров
    lora_dropout: float = 0.05

    #Гиперпараметры обучения
    max_steps: int = 300
    per_device_train_batch_size: int = 1
    gradient_accumulation_steps: int = 8
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.05

    output_dir: str = "outputs-lora-gptj"
    eval_prompts: list = None

cfg = Config()

if cfg.eval_prompts is None:
    cfg.eval_prompts = [
        "Explain what LoRA fine-tuning is in simple words.",
        "Give me three ideas for a weekend hobby for a data scientist.",
        "Summarize this dialogue in one sentence: Alice: how's the status? Bob: shipped v1 yesterday, patch coming Monday.",
        "What is the best way to save money to start a business?"
    ]

cfg


Config(base_model='gpt2', dataset_name='Dahoas/sft-gptj-synthetic-prompt-responses', subset_size=10000, use_qlora=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, max_steps=300, per_device_train_batch_size=1, gradient_accumulation_steps=8, learning_rate=0.0002, warmup_ratio=0.05, output_dir='outputs-lora-gptj', eval_prompts=['Explain what LoRA fine-tuning is in simple words.', 'Give me three ideas for a weekend hobby for a data scientist.', "Summarize this dialogue in one sentence: Alice: how's the status? Bob: shipped v1 yesterday, patch coming Monday.", 'What is the best way to save money to start a business?'])

Токенайзер и базовая модель

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [4]:
# токенайзер
tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# тип чисел
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
bnb_config = None



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
#!pip install -U bitsandbytes accelerate transformers datasets peft trl
cfg.use_qlora = False


In [6]:
if cfg.use_qlora:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=dtype,
    )
    model = AutoModelForCausalLM.from_pretrained(
        cfg.base_model,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        cfg.base_model,
        torch_dtype=dtype,
        device_map="auto",
        trust_remote_code=True,
    )

print("Loaded model:", cfg.base_model)

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loaded model: gpt2


Проверка базовых ответов модели

In [ ]:
#pip install trl

In [9]:
from textwrap import indent
import torch
import re

import inspect
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig


In [10]:
def generate_base(prompt, max_new_tokens=200, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[1]
    with torch.no_grad(): #просто делаем предсказание ничего не обучаем
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
        )
        generated_tokens = outputs[0, input_len:]

    return tokenizer.decode(generated_tokens, skip_special_tokens=True)



In [11]:
cfg.eval_prompts

['Explain what LoRA fine-tuning is in simple words.',
 'Give me three ideas for a weekend hobby for a data scientist.',
 "Summarize this dialogue in one sentence: Alice: how's the status? Bob: shipped v1 yesterday, patch coming Monday.",
 'What is the best way to save money to start a business?']

In [12]:
pre_eval = {}

In [13]:
pre_eval['Explain what LoRA fine-tuning is in simple words.'] = generate_base('Explain what LoRA fine-tuning is in simple words.')

In [14]:
pre_eval['Give me three ideas for a weekend hobby for a data scientist.'] = generate_base('Give me three ideas for a weekend hobby for a data scientist.')

In [15]:
pre_eval["Summarize this dialogue in one sentence: Alice: how's the status? Bob: shipped v1 yesterday, patch coming Monday."] = generate_base("Summarize this dialogue in one sentence: Alice: how's the status? Bob: shipped v1 yesterday, patch coming Monday.")

In [16]:
pre_eval['What is the best way to save money to start a business?'] = generate_base('What is the best way to save money to start a business?')

In [17]:
pre_eval

{'Explain what LoRA fine-tuning is in simple words.': "\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'Yes-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding\n\n* - Check for a 'No-Padding' checkbox if there is no padding",
 'Give me three ideas for a weekend hobby for a data scientist.': '\n\n1. Connect with the Data Scientist and build your skills using data science.\n\n2. Create a data science project that includes all the tools needed to create your own data science project.\n\n3. Create a data science team of data 

In [18]:
#pre_eval = {}
#for p in cfg.eval_prompts:
#    pre_eval[p] = generate_base(p)"

print("=== BASELINE RESPONSES ===")
for p, out in pre_eval.items():
    print("\n# Prompt:\n", p)
    print("\n# Response (baseline):\n", indent(out, "  "))

=== BASELINE RESPONSES ===

# Prompt:
 Explain what LoRA fine-tuning is in simple words.

# Response (baseline):
 

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'Yes-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

# Prompt:
 Give me three ideas for a weekend hobby for a data scientist.

# Response (baseline):
 

  1. Connect with the Data Scientist and build your skills using data science.

  2. Create a data science project that includes all the tools 

In [19]:
ds = load_dataset(cfg.dataset_name)
ds

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

data/train-00000-of-00001-5588d64a331985(…):   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44630 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 44630
    })
})

In [20]:
def format_example(ex):
    # в этом датасете уже есть поля prompt и response
    prompt = ex["prompt"]
    completion = ex["response"]

    # чуть-чуть чистим
    prompt = re.sub(r"\s+$", "", prompt) + "\n"   # убираем пробелы в конце + ровно один \n
    completion = completion.lstrip()             # убираем пробелы в начале

    return {"prompt": prompt, "completion": completion}

def prepare_split(dataset, split="train"):
    data = dataset[split]
    # берём подвыборку, чтобы было быстрее (можно потом увеличить)
    if cfg.subset_size and len(data) > cfg.subset_size:
        data = data.shuffle(seed=42).select(range(cfg.subset_size))
    mapped = data.map(format_example, remove_columns=data.column_names)
    return mapped

train_ds = prepare_split(ds, "train")
print(train_ds[0])
print("Train size:", len(train_ds))


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'prompt': ' What are some coping mechanisms for dealing with depression?\n', 'completion': 'One example of a good way to fight depression is to use a combination of some healthy diet, getting enough sleep, and spending lots of time with friends and family, so that you don’t feel lonely or isolated.  Another good strategy is to engage in some daily activities and hobbies that you enjoy, such as swimming, walking dogs, getting involved in social activities, and spending time outside in the sunshine.  If you’re feeling really down, it’s also a good idea to talk to your doctor about taking medication, either as a prescription drug or a drug like benzodiazepines, to relieve the depression.  Some people find that the symptoms of depression can be greatly reduced by taking certain medications.\n\n'}
Train size: 10000


Настройка LoRa и SFTTrainer

In [21]:
# В какие слои модели вешаем LoRA-адаптеры
#target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
#target_modules = ["q_proj","k_proj","v_proj","o_proj"]
target_modules = ["c_attn", "c_fc", "c_proj"]

# LoRA-конфиг
peft_config = LoraConfig(
    r=cfg.lora_r,
    lora_alpha=cfg.lora_alpha,
    lora_dropout=cfg.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

# Конфиг обучения
sftconf_kwargs = dict(
    output_dir=cfg.output_dir,
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    learning_rate=cfg.learning_rate,
    logging_steps=10,
    max_steps=cfg.max_steps,
    warmup_ratio=cfg.warmup_ratio,
    fp16=not cfg.use_qlora,               # если НЕ QLoRA — используем fp16
    bf16=True if cfg.use_qlora else False,
    optim="paged_adamw_8bit" if cfg.use_qlora else "adamw_torch",
    gradient_checkpointing=True,
    packing=False,
    report_to="none",
)

args = SFTConfig(**sftconf_kwargs)

# Создаём SFTTrainer
trainer_kwargs = dict(
    model=model,
    args=args,
    peft_config=peft_config,
    train_dataset=train_ds,
)

trainer_sig = inspect.signature(SFTTrainer.__init__)
if "processing_class" in trainer_sig.parameters:
    trainer_kwargs["processing_class"] = tokenizer
elif "tokenizer" in trainer_sig.parameters:
    trainer_kwargs["tokenizer"] = tokenizer
if "dataset_text_field" in trainer_sig.parameters:
    trainer_kwargs["dataset_text_field"] = None

trainer = SFTTrainer(**trainer_kwargs)

# Смотрим, сколько параметров реально обучаем
trainable = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)
total = sum(p.numel() for p in trainer.model.parameters())
print(f"Trainable: {trainable:,} / Total: {total:,} ({100*trainable/total:.3f}%)")


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 1024). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainable: 2,359,296 / Total: 126,799,104 (1.861%)


In [22]:
train_result = trainer.train()
train_result

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.197600
20,3.069800
30,3.083600
40,2.874300
50,2.816700
60,2.724900
70,2.670300
80,2.637700
90,2.498400
100,2.711700


TrainOutput(global_step=300, training_loss=2.633639367421468, metrics={'train_runtime': 7232.6899, 'train_samples_per_second': 0.332, 'train_steps_per_second': 0.041, 'total_flos': 161015828281344.0, 'train_loss': 2.633639367421468, 'epoch': 0.24})

Сравнение ответов

In [23]:
trainer.model.eval()
trainer.model.config.use_cache = True

if cfg.use_qlora:
    autocast_dtype = torch.bfloat16
else:
    autocast_dtype = torch.bfloat16   # можно fp16, если удобно

def generate_finetuned(prompt, max_new_tokens=128, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(trainer.model.device)
    with torch.no_grad():
        if torch.cuda.is_available() and autocast_dtype in (torch.float16, torch.bfloat16):
            with torch.autocast("cuda", dtype=autocast_dtype):
                out = trainer.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=temperature,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
        else:
            out = trainer.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
    return tokenizer.decode(out[0], skip_special_tokens=True)



In [24]:
post_eval = {}
for p in cfg.eval_prompts:
    post_eval[p] = generate_finetuned(p)


In [25]:
print("=== POST-TUNING RESPONSES ===")
from textwrap import indent
for p in cfg.eval_prompts:
    print("\n# Prompt:\n", p)
    print("\n# Before:\n", indent(pre_eval[p], "  "))
    print("\n# After:\n", indent(post_eval[p], "  "))

=== POST-TUNING RESPONSES ===

# Prompt:
 Explain what LoRA fine-tuning is in simple words.

# Before:
 

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'Yes-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

  * - Check for a 'No-Padding' checkbox if there is no padding

# After:
   Explain what LoRA fine-tuning is in simple words. 
  LoRA are a group of professionals, who do a lot of work in the legal community, and they are very interested in interpreting the law, and then interpreting the law in a way that is consist

После дообучения с помощью LoRA модель стала генерировать ответы более осмысленно и ближе к формату ответа, чем до обучения, но по качеству видно, что результат все еще далек от адекватного ответа llm: до обучения модель практически давала бессвязный текст, а после — начала пытаться выдавать связанный ответ по теме и пытаться объяснять, хоть и без точного содержания (например, интерпретирует LoRA как юридическую организацию или повторяет части текста). Это говорит о том, что эффект обучения заметен — модель стала лучше реагировать на задачу и формат prompt–response, но из-за маленького размера модели и ограниченного количества обучаемых параметров качество ответов остаётся низким: модель приобрела направление ответа, но не сформировала корректную семантику.
